
# Classification Evaluation Metrics


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime

import random


In [5]:

DEBUG = True
seed = 1919


In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:
#%run "C040_MDL_NN w TensorFlow.ipynb"

In [7]:

#preds_dataset_name = 'predictions_dataset_2022-10-18.csv'
#preds_dataset_name = 'predictions_dataset_2022-10-27.csv'
preds_dataset_name = 'predictions_dataset_2022-11-20.csv'

df_eval_raw = pd.read_csv(f'{fileloc}/{preds_dataset_name}')

print(df_eval_raw.shape)
print()
print(df_eval_raw.columns.values.tolist())


(541552, 59)

['@id_DOG', 'DogName', 'y_runtime', 'RaceId', '@id_RACE', 'Distance', 'RaceGrade', 'Track', 'Event_Dt', 'ft_sp', 'position', 'x_num_dogs', 'Winner_actual', 'Top3_actual', 'position_random', 'Winner_random', 'Top3_random', 'position_fasttrack', 'Winner_fastrack', 'Top3_fastrack', 'y_pos', 'x_dist', 'x_races_tot', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'G5', 'G4', 'G3', 'G2', 'G1', 'Gmix', 'Gmas', 'Gmai', 'Gh', 'Gres', 'Gnov', 'Gsp', 'Gfin', 'RaceGrade_H1', 'x_races_tot_std_no', 'x_races_tot_norm', 'x_speed_prev_std_no', 'x_speed_prev_norm', 'x_speed_avg_std_no', 'x_speed_avg_norm', 'x_speed_min_std_no', 'x_speed_min_norm', 'x_speed_max_std_no', 'x_speed_max_norm', 'x_speed_range_std_no', 'x_speed_range_norm', 'yearqtr', 'ttflag', 'preds']


In [56]:
# Plotting the RaceGrade

In [8]:

print(pd.DataFrame(df_eval_raw.isna().sum()).transpose())


   @id_DOG  DogName  y_runtime  RaceId  @id_RACE  Distance  RaceGrade  Track  Event_Dt  ft_sp  position  x_num_dogs  Winner_actual  Top3_actual  position_random  Winner_random  Top3_random  \
0        0        0          0       0         0         0          0      0         0      0         0           0              0            0                0              0            0   

   position_fasttrack  Winner_fastrack  Top3_fastrack  y_pos  x_dist  x_races_tot  x_no_prev_races  x_speed_prev  ...  G1  Gmix  Gmas  Gmai  Gh  Gres  Gnov  Gsp  Gfin  RaceGrade_H1  \
0                   0                0              0      0       0            0                0             0  ...   0     0     0     0   0     0     0    0     0             0   

   x_races_tot_std_no  x_races_tot_norm  x_speed_prev_std_no  x_speed_prev_norm  x_speed_avg_std_no  x_speed_avg_norm  x_speed_min_std_no  x_speed_min_norm  x_speed_max_std_no  x_speed_max_norm  \
0                   0                 0          

In [9]:

df_eval_raw.describe()


,@id_DOG,y_runtime,RaceId,@id_RACE,ft_sp,position,x_num_dogs,Winner_actual,Top3_actual,position_random,Winner_random,Top3_random,position_fasttrack,Winner_fastrack,Top3_fastrack,y_pos,x_dist,x_races_tot,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,...,G3,G2,G1,Gmix,Gmas,Gmai,Gh,Gres,Gnov,Gsp,Gfin,x_races_tot_std_no,x_races_tot_norm,x_speed_prev_std_no,x_speed_prev_norm,x_speed_avg_std_no,x_speed_avg_norm,x_speed_min_std_no,x_speed_min_norm,x_speed_max_std_no,x_speed_max_norm,x_speed_range_std_no,x_speed_range_norm,yearqtr,preds
count,5.415520e+05,541552.000000,5.415520e+05,5.415520e+05,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,...,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000
mean,4.525647e+08,24.427979,7.292605e+08,7.292605e+08,16.917818,4.150733,7.305492,0.140127,0.419843,4.152746,0.139857,0.419435,4.013938,0.145510,0.437633,4.150733,417.353562,18.734123,0.049223,16258.984439,16372.453807,15870.525764,16780.460082,909.891907,258.242292,...,0.003147,0.001078,0.004627,0.173540,0.021322,0.151780,0.103178,0.053805,0.037082,0.032721,0.042683,0.007656,0.003845,0.006821,0.003427,0.006795,0.003414,0.006795,0.003414,0.006795,0.003414,0.007471,0.003752,202145.009384,3.830250
std,1.121930e+08,4.983891,5.831714e+07,5.831714e+07,20.995473,2.160054,0.950365,0.347119,0.493534,2.159921,0.346839,0.493467,2.088645,0.352614,0.496096,2.160054,83.856726,16.097282,0.216334,3723.162924,3460.682737,3376.122418,3576.492040,705.188284,201.697242,...,0.056006,0.032821,0.067868,0.378714,0.144456,0.358808,0.304191,0.225632,0.188964,0.177905,0.202141,0.087162,0.924927,0.082308,0.925198,0.082153,0.925206,0.082153,0.925206,0.082153,0.925206,0.086112,0.924985,49.218587,0.781730
min,1.083910e+08,10.000000,6.238619e+08,6.238619e+08,0.260000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,259.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.474874,0.000000,-2.474803,0.000000,-2.474852,0.000000,-2.474788,0.000000,-2.474815,0.000000,-2.474874,202101.000000,2.047763
25%,3.836042e+08,20.550000,6.802578e+08,6.802578e+08,4.200000,2.000000,7.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,350.000000,6.000000,0.000000,16795.865633,16881.052752,16306.695464,17237.442922,568.752099,186.539074,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.712055,0.000000,-0.649582,0.000000,-0.672467,0.000000,-0.620811,0.000000,-0.679714,0.000000,-0.678904,202102.000000,3.190755
50%,4.517679e+08,23.540000,7.293859e+08,7.293859e+08,8.700000,4.000000,8.000000,0.000000,0.000000,4.000000,0.000000,0.000000,4.000000,0.000000,0.000000,4.000000,400.000000,15.000000,0.000000,17084.890550,17102.524362,16601.453326,17513.513514,881.521254,250.024868,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.099873,0.000000,0.163156,0.000000,0.067932,0.000000,0.177251,0.000000,0.056865,0.000000,-0.113772,202104.000000,3.773507
75%,5.271657e+08,27.190000,7.792436e+08,7.792436e+08,20.600000,6.000000,8.000000,0.000000,1.000000,6.000000,0.000000,1.000000,6.000000,0.000000,1.000000,6.000000,461.000000,28.000000,0.000000,17347.670251,17287.706370,16842.105263,

In [10]:

df_eval_raw.head()


,@id_DOG,DogName,y_runtime,RaceId,@id_RACE,Distance,RaceGrade,Track,Event_Dt,ft_sp,position,x_num_dogs,Winner_actual,Top3_actual,position_random,Winner_random,Top3_random,position_fasttrack,Winner_fastrack,Top3_fastrack,y_pos,x_dist,x_races_tot,x_no_prev_races,x_speed_prev,...,G1,Gmix,Gmas,Gmai,Gh,Gres,Gnov,Gsp,Gfin,RaceGrade_H1,x_races_tot_std_no,x_races_tot_norm,x_speed_prev_std_no,x_speed_prev_norm,x_speed_avg_std_no,x_speed_avg_norm,x_speed_min_std_no,x_speed_min_norm,x_speed_max_std_no,x_speed_max_norm,x_speed_range_std_no,x_speed_range_norm,yearqtr,ttflag,preds
0,108390981,DOLLY DOES IT,26.43,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,11.0,4,8,0,0,2.0,0,1,5.0,0,0,4,452,0,1,0.0,...,0,1,0,0,0,0,0,0,0,Mixed,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613
1,262316194,GO BEZZA,26.87,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,31.0,8,8,0,0,5.0,0,0,7.0,0,0,8,452,0,1,0.0,...,0,1,0,0,0,0,0,0,0,Mixed,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613
2,285758880,LITTLE MIA,26.81,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,21.0,7,8,0,0,4.0,0,0,6.0,0,0,7,452,0,1,0.0,...,0,1,0,0,0,0,0,0,0,Mixed,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613
3,296096035,MY NAMES SERG,26.13,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,3.8,2,8,0,1,1.0,1,1,2.0,0,1,2,452,0,1,0.0,...,0,1,0,0,0,0,0,0,0,Mixed,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613
4,327377954,WYNBURN BABE,26.01,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,3.0,1,8,1,1,3.0,0,1,1.0,1,1,1,452,0,1,0.0,...,0,1,0,0,0,0,0,0,0,Mixed,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613


In [11]:

df_evaluation = df_eval_raw.copy()


In [12]:

# groupings
groupings = ['@id_RACE']

## Rank for Place Preds
#df_evaluation["position_preds"] = df_evaluation.groupby( groupings )["preds"].rank("dense", ascending=False)
df_evaluation["position_preds"] = df_evaluation.groupby( groupings )["preds"].rank("dense", ascending=True)

# Winner / Place Preds Columns
df_evaluation['Winner_preds'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 1 else 0, axis = 1)
df_evaluation['Top3_preds'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)

print(datetime.now())


2022-11-20 01:43:42.155075


In [13]:

#df_evaluation[df_evaluation['@id_RACE'].isin([626271182])].head()
df_evaluation[df_evaluation['ttflag'].isin(['Test'])].head()


,@id_DOG,DogName,y_runtime,RaceId,@id_RACE,Distance,RaceGrade,Track,Event_Dt,ft_sp,position,x_num_dogs,Winner_actual,Top3_actual,position_random,Winner_random,Top3_random,position_fasttrack,Winner_fastrack,Top3_fastrack,y_pos,x_dist,x_races_tot,x_no_prev_races,x_speed_prev,...,Gmai,Gh,Gres,Gnov,Gsp,Gfin,RaceGrade_H1,x_races_tot_std_no,x_races_tot_norm,x_speed_prev_std_no,x_speed_prev_norm,x_speed_avg_std_no,x_speed_avg_norm,x_speed_min_std_no,x_speed_min_norm,x_speed_max_std_no,x_speed_max_norm,x_speed_range_std_no,x_speed_range_norm,yearqtr,ttflag,preds,position_preds,Winner_preds,Top3_preds
383978,135133731,SOUTHWOOD MIA,20.09,785800389,785800389,340m,Grade 5,Gunnedah,2022-05-26,101.0,5,7,0,0,3.0,0,1,6.0,0,0,5,340,26,0,16666.666667,...,0,0,0,0,0,0,Grade 5,0,0.038648,0,-0.141175,0,-0.787144,0,-2.060100,0,-0.318979,0,1.946134,202202,Test,4.507196,6.0,0,0
383979,403092175,ONLY A WINNER,19.42,785800389,785800389,340m,Grade 5,Gunnedah,2022-05-26,4.8,1,7,1,1,1.0,1,1,2.0,0,1,1,340,32,0,16810.207337,...,0,0,0,0,0,0,Grade 5,0,0.363295,0,0.314235,0,0.567619,0,0.284098,0,0.342085,0,-0.090688,202202,Test,3.368018,2.0,0,1
383980,415622087,LUNAR HALO,20.38,785800389,785800389,340m,Grade 5,Gunnedah,2022-05-26,81.0,7,7,0,0,6.0,0,0,5.0,0,0,7,340,34,0,16640.502355,...,0,0,0,0,0,0,Grade 5,0,0.471510,0,-0.224186,0,0.068881,0,0.288675,0,0.591174,0,0.053048,202202,Test,3.834468,4.0,0,0
383981,448841009,RIVERINA IVY,19.84,785800389,785800389,340m,Grade 5,Gunnedah,2022-05-26,12.0,3,7,0,1,7.0,0,0,3.0,0,1,3,340,58,0,16699.410609,...,0,0,0,0,0,0,Grade 5,0,1.770095,0,-0.037288,0,-0.007469,0,0.026362,0,-0.282868,0,-0.195953,202202,Test,3.969089,5.0,0,0
383982,521874669,MORAINE HALLIE,20.08,785800389,785800389,340m,Grade 5,Gunnedah,2022-05-26,61.0,4,7,0,0,4.0,0,0,4.0,0,0,4,340,15,0,16469.377252,...,0,0,0,0,0,0,Grade 5,0,-0.556537,0,-0.767113,0,0.219676,0,0.365192,0,1.167553,0,0.317280,202202,Test,3.708982,3.0,0,1


In [14]:

df_evaluation.describe()


,@id_DOG,y_runtime,RaceId,@id_RACE,ft_sp,position,x_num_dogs,Winner_actual,Top3_actual,position_random,Winner_random,Top3_random,position_fasttrack,Winner_fastrack,Top3_fastrack,y_pos,x_dist,x_races_tot,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,...,Gmix,Gmas,Gmai,Gh,Gres,Gnov,Gsp,Gfin,x_races_tot_std_no,x_races_tot_norm,x_speed_prev_std_no,x_speed_prev_norm,x_speed_avg_std_no,x_speed_avg_norm,x_speed_min_std_no,x_speed_min_norm,x_speed_max_std_no,x_speed_max_norm,x_speed_range_std_no,x_speed_range_norm,yearqtr,preds,position_preds,Winner_preds,Top3_preds
count,5.415520e+05,541552.000000,5.415520e+05,5.415520e+05,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,...,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000,541552.000000
mean,4.525647e+08,24.427979,7.292605e+08,7.292605e+08,16.917818,4.150733,7.305492,0.140127,0.419843,4.152746,0.139857,0.419435,4.013938,0.145510,0.437633,4.150733,417.353562,18.734123,0.049223,16258.984439,16372.453807,15870.525764,16780.460082,909.891907,258.242292,...,0.173540,0.021322,0.151780,0.103178,0.053805,0.037082,0.032721,0.042683,0.007656,0.003845,0.006821,0.003427,0.006795,0.003414,0.006795,0.003414,0.006795,0.003414,0.007471,0.003752,202145.009384,3.830250,4.097494,0.145735,0.427928
std,1.121930e+08,4.983891,5.831714e+07,5.831714e+07,20.995473,2.160054,0.950365,0.347119,0.493534,2.159921,0.346839,0.493467,2.088645,0.352614,0.496096,2.160054,83.856726,16.097282,0.216334,3723.162924,3460.682737,3376.122418,3576.492040,705.188284,201.697242,...,0.378714,0.144456,0.358808,0.304191,0.225632,0.188964,0.177905,0.202141,0.087162,0.924927,0.082308,0.925198,0.082153,0.925206,0.082153,0.925206,0.082153,0.925206,0.086112,0.924985,49.218587,0.781730,2.149049,0.352841,0.494779
min,1.083910e+08,10.000000,6.238619e+08,6.238619e+08,0.260000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000,259.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-2.474874,0.000000,-2.474803,0.000000,-2.474852,0.000000,-2.474788,0.000000,-2.474815,0.000000,-2.474874,202101.000000,2.047763,1.000000,0.000000,0.000000
25%,3.836042e+08,20.550000,6.802578e+08,6.802578e+08,4.200000,2.000000,7.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000,2.000000,350.000000,6.000000,0.000000,16795.865633,16881.052752,16306.695464,17237.442922,568.752099,186.539074,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.712055,0.000000,-0.649582,0.000000,-0.672467,0.000000,-0.620811,0.000000,-0.679714,0.000000,-0.678904,202102.000000,3.190755,2.000000,0.000000,0.000000
50%,4.517679e+08,23.540000,7.293859e+08,7.293859e+08,8.700000,4.000000,8.000000,0.000000,0.000000,4.000000,0.000000,0.000000,4.000000,0.000000,0.000000,4.000000,400.000000,15.000000,0.000000,17084.890550,17102.524362,16601.453326,17513.513514,881.521254,250.024868,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.099873,0.000000,0.163156,0.000000,0.067932,0.000000,0.177251,0.000000,0.056865,0.000000,-0.113772,202104.000000,3.773507,4.000000,0.000000,0.000000
75%,5.271657e+08,27.190000,7.792436e+08,7.792436e+08,20.600000,6.000000,8.000000,0.000000,1.000000,6.000000,0.000000,1.000000,6.000000,0.000000,1.000000,6.000000,461.000000,28.000000,0.000000,17347.670

In [15]:

df_evaluation.sort_values(by = ['@id_RACE','position']).head()


,@id_DOG,DogName,y_runtime,RaceId,@id_RACE,Distance,RaceGrade,Track,Event_Dt,ft_sp,position,x_num_dogs,Winner_actual,Top3_actual,position_random,Winner_random,Top3_random,position_fasttrack,Winner_fastrack,Top3_fastrack,y_pos,x_dist,x_races_tot,x_no_prev_races,x_speed_prev,...,Gmai,Gh,Gres,Gnov,Gsp,Gfin,RaceGrade_H1,x_races_tot_std_no,x_races_tot_norm,x_speed_prev_std_no,x_speed_prev_norm,x_speed_avg_std_no,x_speed_avg_norm,x_speed_min_std_no,x_speed_min_norm,x_speed_max_std_no,x_speed_max_norm,x_speed_range_std_no,x_speed_range_norm,yearqtr,ttflag,preds,position_preds,Winner_preds,Top3_preds
181390,434274825,FERDINAND BOY,24.98,623861932,623861932,450m,Grade 5 Final,Ballarat,2021-01-01,4.2,1,8,1,1,5.0,0,0,3.0,0,1,1,450,0,1,0.0,...,0,0,0,0,0,1,Grade 5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613,1.0,1,1
181386,380917315,BAD GUY,25.03,623861932,623861932,450m,Grade 5 Final,Ballarat,2021-01-01,5.9,2,8,0,1,1.0,1,1,4.0,0,0,2,450,0,1,0.0,...,0,0,0,0,0,1,Grade 5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613,1.0,1,1
181391,465017181,MASKED AVENGER,25.08,623861932,623861932,450m,Grade 5 Final,Ballarat,2021-01-01,3.2,3,8,0,1,2.0,0,1,1.0,1,1,3,450,0,1,0.0,...,0,0,0,0,0,1,Grade 5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613,1.0,1,1
181384,318755759,ASTON CORINNA,25.32,623861932,623861932,450m,Grade 5 Final,Ballarat,2021-01-01,16.1,4,8,0,0,6.0,0,0,5.0,0,0,4,450,0,1,0.0,...,0,0,0,0,0,1,Grade 5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613,1.0,1,1
181387,387031520,GYPSY ZELDA,25.45,623861932,623861932,450m,Grade 5 Final,Ballarat,2021-01-01,32.1,5,8,0,0,7.0,0,0,8.0,0,0,5,450,0,1,0.0,...,0,0,0,0,0,1,Grade 5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,1,0.5,202101,Train,3.698613,1.0,1,1


In [16]:

pd.crosstab(df_evaluation.position, df_evaluation.position_preds)


position_preds,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0
position,,,,,,,,
1,21160,14992,11802,9454,7546,5532,3516,1884
2,14085,13926,12841,11227,9243,7280,4673,2501
3,11480,12049,12339,11971,10559,8582,5618,3107
4,9591,10550,11444,12107,11522,9835,6737,3722
5,8199,8947,10249,11361,12616,10969,7670,4443
6,6775,7464,8332,9459,11078,12218,8838,5282
7,4737,5357,5885,6544,7569,8847,10658,6312
8,2896,3140,3505,4067,4586,5357,6547,8770


In [17]:

df_evaluation.groupby( [ 'ttflag', 'Top3_actual', 'Top3_preds']).agg({'@id_DOG':'count'}).reset_index()


,ttflag,Top3_actual,Top3_preds,@id_DOG
0,Test,0,0,61214
1,Test,0,1,30338
2,Test,1,0,29960
3,Test,1,1,36062
4,Train,0,0,145900
5,Train,0,1,76733
6,Train,1,0,72733
7,Train,1,1,88612


In [18]:

pd.crosstab(df_evaluation.Top3_actual, df_evaluation.Top3_preds, normalize='all')


Top3_preds,0,1
Top3_actual,,
0,0.382445,0.197711
1,0.189627,0.230216


In [19]:

df_evaluation.tail(10)


,@id_DOG,DogName,y_runtime,RaceId,@id_RACE,Distance,RaceGrade,Track,Event_Dt,ft_sp,position,x_num_dogs,Winner_actual,Top3_actual,position_random,Winner_random,Top3_random,position_fasttrack,Winner_fastrack,Top3_fastrack,y_pos,x_dist,x_races_tot,x_no_prev_races,x_speed_prev,...,Gmai,Gh,Gres,Gnov,Gsp,Gfin,RaceGrade_H1,x_races_tot_std_no,x_races_tot_norm,x_speed_prev_std_no,x_speed_prev_norm,x_speed_avg_std_no,x_speed_avg_norm,x_speed_min_std_no,x_speed_min_norm,x_speed_max_std_no,x_speed_max_norm,x_speed_range_std_no,x_speed_range_norm,yearqtr,ttflag,preds,position_preds,Winner_preds,Top3_preds
541542,667648731,LITTLE FRANKY,23.87,824847265,824847265,400m,Maiden,Gawler,2022-09-23,34.0,2,6,0,1,4.0,0,0,6.0,0,0,2,400,5,0,16977.928693,...,1,0,0,0,0,0,Other,0,2.000000,0,1.279802,0,1.262629,0,1.252321,0,1.279802,0,2.041237,202203,Test,3.839725,2.0,0,1
541543,667700820,DARK PATCH,24.70,824847265,824847265,400m,Maiden,Gawler,2022-09-23,14.0,6,6,0,0,6.0,0,0,4.0,0,0,6,400,0,1,0.000000,...,1,0,0,0,0,0,Other,0,-0.500000,0,-0.645481,0,-0.645395,0,-0.645308,0,-0.645481,0,-0.409289,202203,Test,3.878344,3.0,0,1
541544,679093488,BREE FURY,24.31,824847265,824847265,400m,Maiden,Gawler,2022-09-23,26.0,5,6,0,0,5.0,0,0,5.0,0,0,5,400,0,1,0.000000,...,1,0,0,0,0,0,Other,0,-0.500000,0,-0.645481,0,-0.645395,0,-0.645308,0,-0.645481,0,-0.409289,202203,Test,3.878344,3.0,0,1
541545,702435590,SPRINGVALE CHIP,24.10,824847265,824847265,400m,Maiden,Gawler,2022-09-23,2.9,3,6,0,1,3.0,0,1,2.0,0,1,3,400,1,0,17174.753113,...,1,0,0,0,0,0,Other,0,0.000000,0,1.302122,0,1.318951,0,1.328911,0,1.302122,0,-0.404080,202203,Test,2.731178,1.0,1,1
541546,692849434,CADILLAC SEDAN,18.50,824845033,824845033,305m,Maiden,Mount Gambier,2022-09-18,6.5,4,6,0,0,1.0,1,1,4.0,0,0,4,305,0,1,0.000000,...,1,0,0,0,0,0,Other,0,-0.645497,0,-0.645447,0,-0.645447,0,-0.645447,0,-0.645447,0,-0.645497,202203,Test,3.871552,3.0,0,1
541547,692849435,CADILLAC STAR,17.93,824845033,824845033,305m,Maiden,Mount Gambier,2022-09-18,2.7,1,6,1,1,6.0,0,0,1.0,1,1,1,305,1,0,17039.106145,...,1,0,0,0,0,0,Other,0,1.290994,0,1.310701,0,1.310701,0,1.310701,0,1.310701,0,1.290994,202203,Test,3.301083,1.0,1,1
541548,692849436,CADILLAC COUPE,18.50,824845033,824845033,305m,Maiden,Mount Gambier,2022-09-18,7.5,5,6,0,0,2.0,0,1,5.0,0,0,5,305,0,1,0.000000,...,1,0,0,0,0,0,Other,0,-0.645497,0,-0.645447,0,-0.645447,0,-0.645447,0,-0.645447,0,-0.645497,202203,Test,3.871552,3.0,0,1
541549,692849437,CADILLAC LLOYD,18.24,824845033,824845033,305m,Maiden,Mount Gambier,2022-09-18,5.0,2,6,0,1,3.0,0,1,3.0,0,1,2,305,0,1,0.000000,...,1,0,0,0,0,0,Other,0,-0.645497,0,-0.645447,0,-0.645447,0,-0.645447,0,-0.645447,0,-0.645497,202203,Test,3.871552,3.0,0,1
541550,692849438,CADILLAC MAGIC,18.69,824845033,824845033,305m,Maiden,Mount Gambier,2022-09-18,12.0,6,6,0,0,4.0,0,0,6.0,0,0,6,305,1,0,16694.033935,...,1,0,0,0,0,0,Other,0,1.290994,0,1.271085,0,1.271085,0,1.271085,0,1.271085,0,1.290994,202203,Test,3.353467,2.0,0,1
541551,695984743,COMPTON BUSTER,18.38,824845033,824845033,305m,Maiden,Mount Gambier,2022-09-18,3.7,3,6,0,1,5.0,0,0,2.0,0,1,3,305,0,1,0.000000,...,1,0,0,0,0,0,Other,0,-0.645497,0,-0.645447,0,-0.645447,0,-0.645447,0,-0.645447,0,-0.645497,202203,Test,3.871552,3.0,0,1


In [20]:

print(datetime.now())


2022-11-20 01:43:43.249310


In [21]:

# Random


In [22]:


# Bet on all Preds = s1
df_evaluation['rand_1'] = df_evaluation.apply( lambda x : True if x.position_random == 1 else False, axis = 1 )
df_evaluation['h_rand_1'] = df_evaluation.apply( lambda x : 1 if ( x.rand_1 == 1 ) & ( x.position == 1 ) \
                                             else 0 if (x.rand_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )
df_evaluation['p_rand_1'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.rand_1 == 1 ) & ( x.position == 1 ) \
                                             else -1 if (x.rand_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )


# Bet on all Place = s2
#df_evaluation['fs_2'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)
df_evaluation["rand_all_plc"] = df_evaluation.apply(lambda x : np.nan if pd.isna(x.position_random) \
                                                                else True if (x.x_num_dogs >= 8) & (x.position_random <= 3) \
                                                                    else True if (x.x_num_dogs >= 5) & (x.position_random <= 2) \
                                                                        else False if (x.x_num_dogs < 5) \
                                                                            else False, axis = 1)
df_evaluation['h_rand_all_plc'] = df_evaluation.apply( lambda x : 1 if ( x.rand_all_plc == 1 ) & ( x.position == 1 ) \
                                             else 0 if ( x.rand_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_evaluation['p_rand_all_plc'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.rand_all_plc == 1 ) & ( x.position == 1 ) \
                                             else -1 if ( x.rand_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

print()

print(datetime.now())



2022-11-20 01:44:49.548627


In [23]:

# Benchmark


In [24]:

# Bet on all Preds = s1
df_evaluation['ft_1'] = df_evaluation.apply( lambda x : True if x.position_fasttrack == 1 else False, axis = 1 )
df_evaluation['h_ft_1'] = df_evaluation.apply( lambda x : 1 if ( x.ft_1 == 1 ) & ( x.position == 1 ) \
                                             else 0 if (x.ft_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )
df_evaluation['p_ft_1'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.ft_1 == 1 ) & ( x.position == 1 ) \
                                             else -1 if (x.ft_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )


# Bet on all Place = s2
#df_evaluation['fs_2'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)
df_evaluation["ft_all_plc"] = df_evaluation.apply(lambda x : np.nan if pd.isna(x.position_fasttrack) \
                                                                else True if (x.x_num_dogs >= 8) & (x.position_fasttrack <= 3) \
                                                                    else True if (x.x_num_dogs >= 5) & (x.position_fasttrack <= 2) \
                                                                        else False if (x.x_num_dogs < 5) \
                                                                            else False, axis = 1)
df_evaluation['h_ft_all_plc'] = df_evaluation.apply( lambda x : 1 if ( x.ft_all_plc == 1 ) & ( x.position == 1 ) \
                                             else 0 if ( x.ft_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_evaluation['p_ft_all_plc'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.ft_all_plc == 1 ) & ( x.position == 1 ) \
                                             else -1 if ( x.ft_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

print()

print(datetime.now())
#3Min runtime



2022-11-20 01:45:56.059710


In [25]:

# Strategies


In [26]:

# Bet on all Preds = s1
df_evaluation['s_1'] = df_evaluation.apply( lambda x : True if x.position_preds == 1 else False, axis = 1 )
df_evaluation['h_s_1'] = df_evaluation.apply( lambda x : 1 if ( x.s_1 == 1 ) & ( x.position == 1 ) \
                                             else 0 if (x.s_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )
df_evaluation['p_s_1'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.s_1 == 1 ) & ( x.position == 1 ) \
                                             else -1 if (x.s_1 == 1) & ( x.position > 1 )\
                                             else 0, axis = 1 )


# Bet on all Place = s2
#df_evaluation['fs_2'] = df_evaluation.apply(lambda x : 1 if x.position_preds <= 3 else 0, axis = 1)
df_evaluation["s_all_plc"] = df_evaluation.apply(lambda x : np.nan if pd.isna(x.position_preds) \
                                                                else True if (x.x_num_dogs >= 8) & (x.position_preds <= 3) \
                                                                    else True if (x.x_num_dogs >= 5) & (x.position_preds <= 2) \
                                                                        else False if (x.x_num_dogs < 5) \
                                                                            else False, axis = 1)

df_evaluation['h_s_all_plc'] = df_evaluation.apply( lambda x : 1 if ( x.s_all_plc == 1 ) & ( x.position == 1 ) \
                                             else 0 if ( x.s_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

df_evaluation['p_s_all_plc'] = df_evaluation.apply( lambda x : x.ft_sp - 1 if ( x.s_all_plc == 1 ) & ( x.position == 1 ) \
                                             else -1 if ( x.s_all_plc == 1 ) & ( x.position > 1 ) \
                                             else 0 , axis = 1 )

print()

print(datetime.now())
#3min runtime



2022-11-20 01:47:02.698097


In [27]:

df_evaluation.head()


,@id_DOG,DogName,y_runtime,RaceId,@id_RACE,Distance,RaceGrade,Track,Event_Dt,ft_sp,position,x_num_dogs,Winner_actual,Top3_actual,position_random,Winner_random,Top3_random,position_fasttrack,Winner_fastrack,Top3_fastrack,y_pos,x_dist,x_races_tot,x_no_prev_races,x_speed_prev,...,x_speed_range_norm,yearqtr,ttflag,preds,position_preds,Winner_preds,Top3_preds,rand_1,h_rand_1,p_rand_1,rand_all_plc,h_rand_all_plc,p_rand_all_plc,ft_1,h_ft_1,p_ft_1,ft_all_plc,h_ft_all_plc,p_ft_all_plc,s_1,h_s_1,p_s_1,s_all_plc,h_s_all_plc,p_s_all_plc
0,108390981,DOLLY DOES IT,26.43,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,11.0,4,8,0,0,2.0,0,1,5.0,0,0,4,452,0,1,0.0,...,0.5,202101,Train,3.698613,1.0,1,1,False,0,0.0,True,0,-1.0,False,0,0.0,False,0,0.0,True,0,-1.0,True,0,-1.0
1,262316194,GO BEZZA,26.87,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,31.0,8,8,0,0,5.0,0,0,7.0,0,0,8,452,0,1,0.0,...,0.5,202101,Train,3.698613,1.0,1,1,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0,True,0,-1.0,True,0,-1.0
2,285758880,LITTLE MIA,26.81,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,21.0,7,8,0,0,4.0,0,0,6.0,0,0,7,452,0,1,0.0,...,0.5,202101,Train,3.698613,1.0,1,1,False,0,0.0,False,0,0.0,False,0,0.0,False,0,0.0,True,0,-1.0,True,0,-1.0
3,296096035,MY NAMES SERG,26.13,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,3.8,2,8,0,1,1.0,1,1,2.0,0,1,2,452,0,1,0.0,...,0.5,202101,Train,3.698613,1.0,1,1,True,0,-1.0,True,0,-1.0,False,0,0.0,True,0,-1.0,True,0,-1.0,True,0,-1.0
4,327377954,WYNBURN BABE,26.01,626271182,626271182,452m,Mixed 4/5,Devonport,2021-01-05,3.0,1,8,1,1,3.0,0,1,1.0,1,1,1,452,0,1,0.0,...,0.5,202101,Train,3.698613,1.0,1,1,False,0,0.0,True,1,2.0,True,1,2.0,True,1,2.0,True,1,2.0,True,1,2.0


In [28]:

pd.set_option('display.max_rows', 50)


In [29]:

# Correlations


In [30]:

print('Position vs Random')
print(df_evaluation['position'].corr( df_evaluation['position_random'] ))
print()

print('Position vs Preds')
print(df_evaluation['position'].corr( df_evaluation['position_preds'] ))
print()

print('Position vs FastTrack')
print(df_evaluation['position'].corr( df_evaluation['position_fasttrack'] ))

print()
print()
print('Preds vs Fasttrack')
print(df_evaluation['position_preds'].corr( df_evaluation['position_fasttrack'] ))


Position vs Random
0.04730107631741168

Position vs Preds
0.29628264328800974

Position vs FastTrack
0.46408294179279436


Preds vs Fasttrack
0.4835335507296271


In [31]:

# Classic overestimation of Placement problems -- placing top 3 for 8, and also 3 for 7,6,5,4,3,2,1 racers (fix)
# Classic using WIN price for PLACE issue

df_eval = df_evaluation.groupby(['ttflag'])\
.agg({'ft_1':'sum', 'h_ft_1':'sum', 'p_ft_1':'sum', 'ft_all_plc':'sum', 'h_ft_all_plc':'sum', 'p_ft_all_plc':'sum'\
      , 's_1':'sum', 'h_s_1':'sum', 'p_s_1' :'sum' ,'s_all_plc':'sum', 'h_s_all_plc':'sum', 'p_s_all_plc':'sum'\
      , 'rand_1':'sum', 'h_rand_1':'sum', 'p_rand_1' :'sum' ,'rand_all_plc':'sum', 'h_rand_all_plc':'sum', 'p_rand_all_plc':'sum'\
      ,'@id_RACE':'nunique' })\
.reset_index()\
.transpose()


In [32]:

df_eval


,0,1
ttflag,Test,Train
ft_1,22841,55960
h_ft_1,9384,22788
p_ft_1,-3000.4,-7902.68
ft_all_plc,57095,139070
h_ft_all_plc,15723,38326
p_ft_all_plc,-9607.01,-22279.1
s_1,22050,56873
h_s_1,6258,14902
p_s_1,-2621.04,-8382.52


In [34]:

#df_eval.to_csv(f'{fileloc}/evaluation_2022-10-18_extended_totals.csv', index = True)
#df_eval.to_csv(f'{fileloc}/evaluation_2022-10-27_extended_totals.csv', index = True)
df_eval.to_csv(f'{fileloc}/evaluation_2022-11-20_extended_totals.csv', index = True)


In [35]:

df_res = df_evaluation.groupby(['ttflag','Track'])\
.agg({'ft_1':'sum', 'h_ft_1':'sum', 'p_ft_1':'sum', 'ft_all_plc':'sum', 'h_ft_all_plc':'sum', 'p_ft_all_plc':'sum'\
      , 's_1':'sum', 'h_s_1':'sum', 'p_s_1' :'sum' ,'s_all_plc':'sum', 'h_s_all_plc':'sum', 'p_s_all_plc':'sum'\
      , 'rand_1':'sum', 'h_rand_1':'sum', 'p_rand_1' :'sum' ,'rand_all_plc':'sum', 'h_rand_all_plc':'sum', 'p_rand_all_plc':'sum'\
      ,'@id_RACE':'nunique' })\
.reset_index()\
.transpose()


In [36]:
df_res

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,...,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109
ttflag,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,Test,...,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train,Train
Track,Albion Park,Angle Park,Ballarat,Bathurst,Bendigo,Broken Hill,Bulli,Bundaberg,Cannington,Capalaba,Casino,Coonamble,Dapto,Darwin,Dubbo,Gawler,Geelong,Gosford,Goulburn,Grafton,Gunnedah,Healesville,Hobart,Horsham,Ipswich,...,Mandurah,Meadows,Moree,Mount Gambier,Murray Bridge,Muswellbrook,Nowra,Potts Park,Richmond,Rockhampton,Sale,Sandown Park,Shepparton,Tamworth,Taree,Temora,The Gardens,Townsville,Traralgon,Wagga,Warragul,Warrnambool,Wauchope,Wentworth Park,Young
ft_1,911,750,705,254,682,100,243,152,999,408,453,53,253,349,259,623,669,310,290,397,282,705,257,378,1110,...,2533,1391,114,1413,1586,74,566,86,2408,671,1115,1320,1659,171,401,409,1530,1265,210,541,1874,1453,147,1066,134
h_ft_1,382,314,284,125,258,44,103,72,389,204,173,19,96,149,105,264,287,127,129,169,114,276,96,167,440,...,1005,565,31,606,760,26,243,23,991,263,461,510,642,68,142,171,611,474,89,206,779,566,56,421,29
p_ft_1,-61.86,-106.55,-85.36,4.79,-140.2,-15.98,-47.43,1.2,-83.52,-18.53,-68.27,-17.95,-44.83,-41.6,-54.78,-99.99,-57.52,-31.69,-39.15,-47.65,-40.01,-134.98,-57.41,-60.08,-147.9,...,-311.16,-197.28,-63.2,-143.54,-166.24,-20.7,-59.15,-38.55,-288.61,-98.85,-141.9,-235.54,-290.76,-28.3,-77.3,-58.03,-180.96,-184.74,-23.5,-59.49,-212.38,-228.12,-23.2,-161,-68.4
ft_all_plc,2474,1639,1815,632,1746,213,559,385,2534,1093,1231,120,638,793,659,1352,1608,806,731,999,779,1778,706,882,3020,...,6700,3314,181,2906,3218,185,1316,209,6597,1845,2612,3212,4271,427,1098,1065,4163,3341,485,1449,4649,3649,374,2791,357
h_ft_all_plc,654,484,488,179,463,61,175,115,670,317,318,30,173,218,169,416,452,213,209,296,195,496,176,264,803,...,1786,911,43,904,1061,47,390,46,1750,489,729,910,1132,116,258,282,1078,852,137,369,1308,1022,100,747,69
p_ft_all_plc,-404.89,-222.95,-299.36,-157.31,-280.1,-51.08,-89.43,-37.45,-338.56,-256.83,-223.72,-45.25,-84.56,-167.6,-187.88,-209.79,-250.16,-161.34,-136.7,-127.35,-192.16,-247.08,-156.91,-157.08,-513.85,...,-922.96,-619,-84.5,-414.04,-472.13,-48.9,-190.8,-86.35,-1143.58,-280.98,-423.7,-313.34,-689.16,-128.4,-252.4,-209.33,-795.1,-593.36,-95,-225.67,-617.12,-417.32,-47.85,-374.7,-164.05
s_1,892,733,690,245,669,90,237,147,976,397,443,49,247,337,249,609,662,304,287,394,278,687,263,373,1075,...,2604,1459,61,1410,1650,62,612,73,2461,714,1137,1371,1657,151,388,399,1629,1293,199,591,1978,1523,176,1100,97
h_s_1,181,212,217,80,196,28,72,39,278,107,139,22,68,120,84,176,188,75,89,111,82,191,66,120,257,...,728,342,23,419,498,15,165,14,658,181,325,319,449,45,107,113,391,334,71,134,534,438,49,262,23


In [37]:

#df_res.to_csv(f'{fileloc}/evaluation_2022-10-18_extended_bytrack.csv', index = True)
#df_res.to_csv(f'{fileloc}/evaluation_2022-10-27_extended_bytrack.csv', index = True)
df_res.to_csv(f'{fileloc}/evaluation_2022-11-20_extended_bytrack.csv', index = True)


In [49]:

print("THE END")


THE END
